<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/tf_quant_finance_notebooks/hull_white_calibration_from_cap_floors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## tf_quant_finance.models.hull_white.calibration_from_swaptions

The Hull-White one-factor model assumes that the short rate $r(t)$ evolves according to the stochastic differential equation:
$$
dr(t) = \left[\theta(t) - a r(t)\right] dt + \sigma dW(t),
$$
where:
- $a$: mean-reversion speed,
- $\sigma$: volatility of the short rate,
- $\theta(t)$: time-dependent drift, calibrated to fit the initial term structure,
- $W(t)$: standard Brownian motion under the risk-neutral measure.

This function estimates the mean-reversion rate and volatility parameters of a Hull-White 1-factor model using a set of European swaption prices as the target. The calibration is performed using least-squares optimization where the loss function minimizes the squared error between the target swaption prices and the model implied swaption prices.

### Example
The example shows how to calibrate a Hull-White model with constant mean reversion rate and constant volatility.

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [ ]:
!pip install tf-quant-finance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow.compat.v2 as tf
import tf_quant_finance as tff

dtype = tf.float64

mean_reversion = [0.03]
volatility = [0.01]
expiries = np.array(
    [0.5, 0.5, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0, 4.0, 4.0, 5.0, 5.0, 10., 10.])
float_leg_start_times = np.array([
    [0.5, 1.0, 1.5, 2.0, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5],  # 6M x 2Y  swap
    [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],  # 6M x 5Y  swap
    [1.0, 1.5, 2.0, 2.5, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],  # 1Y x 2Y  swap
    [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5],  # 1Y x 5Y  swap
    [2.0, 2.5, 3.0, 3.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0],  # 2Y x 2Y  swap
    [2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5],  # 2Y x 5Y  swap
    [3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0],  # 3Y x 2Y  swap
    [3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5],  # 3Y x 5Y  swap
    [4.0, 4.5, 5.0, 5.5, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0],  # 4Y x 2Y  swap
    [4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5],  # 4Y x 5Y  swap
    [5.0, 5.5, 6.0, 6.5, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0],  # 5Y x 2Y  swap
    [5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5],  # 5Y x 5Y  swap
    [10.0, 10.5, 11.0, 11.5, 12.0, 12.0, 12.0, 12.0, 12.0,
     12.0],  # 10Y x 2Y  swap
    [10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0,
     14.5]  # 10Y x 5Y  swap
])
float_leg_end_times = float_leg_start_times + 0.5
max_maturities = np.array(
    [2.5, 5.5, 3.0, 6.0, 4., 7., 5., 8., 6., 9., 7., 10., 12., 15.])
for i in range(float_leg_end_times.shape[0]):
  float_leg_end_times[i] = np.clip(
      float_leg_end_times[i], 0.0, max_maturities[i])

fixed_leg_payment_times = float_leg_end_times
float_leg_daycount_fractions = (
    float_leg_end_times - float_leg_start_times)
fixed_leg_daycount_fractions = float_leg_daycount_fractions
fixed_leg_coupon = 0.01 * np.ones_like(fixed_leg_payment_times)

zero_rate_fn = lambda x: 0.01 * tf.ones_like(x, dtype=dtype)
prices = tff.models.hull_white.swaption_price(
    expiries=expiries,
    floating_leg_start_times=float_leg_start_times,
    floating_leg_end_times=float_leg_end_times,
    fixed_leg_payment_times=fixed_leg_payment_times,
    floating_leg_daycount_fractions=float_leg_daycount_fractions,
    fixed_leg_daycount_fractions=fixed_leg_daycount_fractions,
    fixed_leg_coupon=fixed_leg_coupon,
    reference_rate_fn=zero_rate_fn,
    notional=100.,
    mean_reversion=mean_reversion,
    volatility=volatility,
    use_analytic_pricing=True,
    dtype=dtype)

calibrated_parameters = tff.models.hull_white.calibration_from_swaptions(
    prices=prices,
    expiries=expiries,
    floating_leg_start_times=float_leg_start_times,
    floating_leg_end_times=float_leg_end_times,
    fixed_leg_payment_times=fixed_leg_payment_times,
    floating_leg_daycount_fractions=float_leg_daycount_fractions,
    fixed_leg_daycount_fractions=fixed_leg_daycount_fractions,
    fixed_leg_coupon=fixed_leg_coupon,
    reference_rate_fn=zero_rate_fn,
    notional=100.,
    mean_reversion=[0.01],  # Initial guess for mean reversion rate
    volatility=[0.005],  # Initial guess for volatility
    maximum_iterations=50,
    dtype=dtype)
# Expected calibrated_parameters.mean_reversion.values(): [0.03]
# Expected calibrated_parameters.volatility.values(): [0.01]

In [ ]:
from tensorflow.python.framework.tensor_util import MakeNdarray

# Unpack the tuple
calibration_result, bool_tensor, int_tensor = calibrated_parameters

# Extract values
mean_reversion = calibration_result.mean_reversion
volatility = calibration_result.volatility
bool_value = bool_tensor.numpy()
int_value = int_tensor.numpy()

# Print the extracted values
print("Mean Reversion:", mean_reversion.values().numpy())
print("Volatility:", volatility.values().numpy())
print("Boolean Value:", bool_value)
print("Integer Value:", int_value)

Mean Reversion: [0.03000001]
Volatility: [0.01]
Boolean Value: True
Integer Value: 14
